In [43]:
import json
import numpy as np
import pandas as pd
from dateutil import parser
import preprocess_hop_log, merge_hop_tick, aggregate_rows_by_time
import os

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [44]:
# path_hoplog = '../raw_data/hopLogger/12/'
# path_ticklog = '../raw_data/tickLogger/12/'
# path_mergeddf = '../preprocessed_data/12/'

DIR_NAME = 176

path_hoplog = '../raw_data/hopLogger/' + str(DIR_NAME) + '/'
path_ticklog = '../raw_data/tickLogger/' + str(DIR_NAME) + '/'
# path_hoplog = '../../wowsan/log/hopLogger/' + str(DIR_NAME) + '/'
# path_ticklog = '../../wowsan/log/tickLogger/' + str(DIR_NAME) + '/'
path_mergeddf = '../preprocessed_data_slotted/' + str(DIR_NAME) + '/'


In [45]:
NUM_BROKERS = 5
BROKER_PORT = 50002

## Hop Log 전처리

In [46]:
# json 파일을 열고 각 줄을 개별적으로 파싱
# path = '../raw_data/hopLogger/116/'
data = []
with open(path_hoplog + f'{BROKER_PORT}.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

# 데이터를 DataFrame으로 변환
hop_df_raw = pd.DataFrame(data)
type(hop_df_raw)

pandas.core.frame.DataFrame

In [47]:
hop_df_raw

,Node,PerformanceInfo,level,msg,time
0,localhost:50002,"[{'BrokerId': 'localhost:50003', 'Cpu': '58.481087', 'Memory': '22659072', 'QueueLength': '0', 'QueueTime': '0.109688', 'AverageQueueTime': '', 'ServiceTime': '0.000000', 'AverageServiceTime': '', 'ResponseTime': '0.109688', 'InterArrivalTime': '0.000000', 'Throughput': '9.116759', 'AverageThroughput': '', 'Timestamp': '2024-11-10 18:10:22.0186605 +0900 KST m=+5.707780101'}, {'BrokerId': 'localhost:50002', 'Cpu': '58.342480', 'Memory': '23302144', 'QueueLength': '0', 'QueueTime': '0.300874', 'AverageQueueTime': '0.300874', 'ServiceTime': '0.000000', 'AverageServiceTime': '0.000000', 'ResponseTime': '0.300874', 'InterArrivalTime': '0.000000', 'Throughput': '3.323645', 'AverageThroughput': '3.323645', 'Timestamp': '2024-11-10 18:10:22.6333787 +0900 KST m=+6.682471101'}]",info,Advertisement(apple > 0.45029967260592724) c1dd8768-b009-454c-8b86-6cfb88d05c89,2024-11-10 18:10:22
1,localhost:50002,"[{'BrokerId': 'localhost:50003', 'Cpu': '61.388747', 'Memory': '23334912', 'QueueLength': '0', 'QueueTime': '0.111847', 'AverageQueueTime': '', 'ServiceTime': '0.110225', 'AverageServiceTime': '', 'ResponseTime': '0.222072', 'InterArrivalTime': '0.774395', 'Throughput': '4.503044', 'AverageThroughput': '', 'Timestamp': '2024-11-10 18:10:24.344005 +0900 KST m=+8.033124601'}, {'BrokerId': 'localhost:50002', 'Cpu': '56.639821', 'Memory': '23822336', 'QueueLength': '0', 'QueueTime': '0.304711', 'AverageQueueTime': '0.302793', 'ServiceTime': '0.932874', 'AverageServiceTime': '0.932874', 'ResponseTime': '1.237585', 'InterArrivalTime': '1.174595', 'Throughput': '0.808025', 'AverageThroughput': '0.809280', 'Timestamp': '2024-11-10 18:10:24.9864043 +0900 KST m=+9.035496701'}]",info,Advertisement(apple > 0.6372624597291875) 6585742f-dcf6-4d9e-aaed-40b7fae5df07,2024-11-10 18:10:24
2,localhost:50002,"[{'BrokerId': 'localhost:50003', 'Cpu': '66.056112', 'Memory': '23732224', 'QueueLength': '0', 'QueueTime': '0.109257', 'AverageQueueTime': '', 'ServiceTime': '0.105346', 'AverageServiceTime': '', 'ResponseTime': '0.214603', 'InterArrivalTime': '0.942848', 'Throughput': '4.659758', 'AverageThroughput': '', 'Timestamp': '2024-11-10 18:10:26.7324723 +0900 KST m=+10.421591901'}, {'BrokerId': 'localhost:50002', 'Cpu': '63.944845', 'Memory': '24092672', 'QueueLength': '0', 'QueueTime': '0.323526', 'AverageQueueTime': '0.309704', 'ServiceTime': '0.986176', 'AverageServiceTime': '0.959525', 'ResponseTime': '1.309703', 'InterArrivalTime': '1.580680', 'Throughput': '0.763532', 'AverageThroughput': '0.787880', 'Timestamp': '2024-11-10 18:10:27.3980715 +0900 KST m=+11.447163901'}]",info,Advertisement(apple > 0.7089373080942825) 458907d5-309f-4737-8dbe-570f0d2abb36,2024-11-10 18:10:27
3,localhost:50002,"[{'BrokerId': 'localhost:50003', 'Cpu': '65.593299', 'Memory': '24068096', 'QueueLength': '0', 'QueueTime': '0.111773', 'AverageQueueTime': '', 'ServiceTime': '0.120592', 'AverageServiceTime': '', 'ResponseTime': '0.232365', 'InterArrivalTime': '1.034128', 'Throughput': '4.303582', 'AverageThroughput': '', 'Timestamp': '2024-11-10 18:10:29.2596407 +0900 KST m=+12.948760301'}, {'BrokerId': 'localhost:50002', 'Cpu': '59.767774', 'Memory': '24530944', 'QueueLength': '0', 'QueueTime': '0.303518', 'AverageQueueTime': '0.308157', 'ServiceTime': '1.139734', 'AverageServiceTime': '1.019595', 'ResponseTime': '1.443252', 'InterArrivalTime': '1.835172', 'Throughput': '0.692880', 'AverageThroughput': '0.753152', 'Timestamp': '2024-11-10 18:10:29.976711 +0900 KST m=+14.025803401'}]",info,Advertisement(apple > 0.21606409572158647) 5c6d10fb-167e-49c2-a606-204af37c7a25,2024-11-10 18:10:29
4,localhost:50002,"[{'BrokerId': 'localhost:50003', 'Cpu': '58.115360', 'Memory': '24485888', 'QueueLength': '0', 'QueueTime': '0.106496', 'AverageQueueTime': '', 'ServiceTime': '0.118152', 'AverageServiceTime': '', 'ResponseTime': '0.224648', 'InterArrivalTime': '1.118635', 'Throughput': '4.451416', 'AverageThroughput': '', 'Timestamp': '2024-1

In [48]:
# hop_df_raw 타입 확인
hop_df_raw.dtypes

Node               object
PerformanceInfo    object
level              object
msg                object
time               object
dtype: object

In [49]:
hop_df = preprocess_hop_log.parse_performance_info(NUM_BROKERS, hop_df_raw)

In [50]:
hop_df

,Node,level,msg,time,B1_Cpu,B1_Memory,B1_QueueLength,B1_QueueTime,B1_ServiceTime,B1_ResponseTime,B1_InterArrivalTime,B1_Throughput,B1_m,B1_TimestampDiff,B2_Cpu,B2_Memory,B2_QueueLength,B2_QueueTime,B2_ServiceTime,B2_ResponseTime,B2_InterArrivalTime,B2_Throughput,B2_m,B2_TimestampDiff,B3_Cpu,B3_Memory,B3_QueueLength,B3_QueueTime,B3_ServiceTime,B3_ResponseTime,B3_InterArrivalTime,B3_Throughput,B3_m,B3_TimestampDiff,B4_Cpu,B4_Memory,B4_QueueLength,B4_QueueTime,B4_ServiceTime,B4_ResponseTime,B4_InterArrivalTime,B4_Throughput,B4_m,B4_TimestampDiff,B5_Cpu,B5_Memory,B5_QueueLength,B5_QueueTime,B5_ServiceTime,B5_ResponseTime,B5_InterArrivalTime,B5_Throughput,B5_m,B5_TimestampDiff
0,localhost:50002,info,Advertisement(apple > 0.45029967260592724) c1dd8768-b009-454c-8b86-6cfb88d05c89,2024-11-10 18:10:22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.342480,23302144.0,0.0,0.300874,0.000000,0.300874,0.000000,3.323645,6.682471,NaN,58.481087,22659072.0,0.0,0.109688,0.000000,0.109688,0.000000,9.116759,5.707780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,localhost:50002,info,Advertisement(apple > 0.6372624597291875) 6585742f-dcf6-4d9e-aaed-40b7fae5df07,2024-11-10 18:10:24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.639821,23822336.0,0.0,0.304711,0.932874,1.237585,1.174595,0.808025,9.035497,NaN,61.388747,23334912.0,0.0,0.111847,0.110225,0.222072,0.774395,4.503044,8.033125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,localhost:50002,info,Advertisement(apple > 0.7089373080942825) 458907d5-309f-4737-8dbe-570f0d2abb36,2024-11-10 18:10:27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.944845,24092672.0,0.0,0.323526,0.986176,1.309703,1.580680,0.763532,11.447164,NaN,66.056112,23732224.0,0.0,0.109257,0.105346,0.214603,0.942848,4.659758,10.421592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,localhost:50002,info,Advertisement(apple > 0.21606409572158647) 5c6d10fb-167e-49c2-a606-204af37c7a25,2024-11-10 18:10:29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.767774,24530944.0,0.0,0.303518,1.139734,1.443252,1.835172,0.692880,14.025803,NaN,65.593299,24068096.0,0.0,0.111773,0.120592,0.232365,1.034128,4.303582,12.948760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,localhost:50002,info,Advertisement(apple > 0.8015435687024113) 9108cdbc-fe94-47b2-86c0-7a88d7c6aa81,2024-11-10 18:10:32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.645620,24924160.0,0.0,0.317207,1.185807,1.503013,2.053788,0.665330,16.967743,NaN,58.115360,24485888.0,0.0,0.106496,0.118152,0.224648,1.118635,4.451416,15.772307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3497,localhost:50002,info,Advertisement(apple > 0.6517608812353198) bc7adf86-ad7b-4496-a87d-4dacf3637019,2024-11-10 20:09:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.377076,71446528.0,0.0,0.014223,1.788314,1.802537,2.048900,0.554774,7173.449767,NaN,57.389806,134955008.0,71414.0,6525.396436,2.354128,6527.750564,0.041454,0.000153,7171.015352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3498,localhost:50002,info,Advertisement(apple > 0.8133929250479679) bee384b6-0055-45a3-a8f7-881a7bd8baef,2024-11-10 20:09:54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.379479,41439232.0,0.0,0.001527,2.444717,2.446244,2.049682,0.408790,7178.221656,NaN,57.395523,144584704.0,71412.0,6529.669858,2.183997,6531.853855,0.041454,0.000153,7175.364701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3499,localhost:50002,info,Advertisement(apple > 0.6941225650552122) 6296a3e5-7221-468f-aa80-50b195a238e1,2024-11-10 20:10:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.387428,42385408.0,0.0,0.016

In [51]:
# 컬럼별 데이터 타입 확인
hop_df.dtypes

Node                    object
level                   object
msg                     object
time                    object
B1_Cpu                 float64
B1_Memory              float64
B1_QueueLength         float64
B1_QueueTime           float64
B1_ServiceTime         float64
B1_ResponseTime        float64
B1_InterArrivalTime    float64
B1_Throughput          float64
B1_m                   float64
B1_TimestampDiff       float64
B2_Cpu                 float64
B2_Memory              float64
B2_QueueLength         float64
B2_QueueTime           float64
B2_ServiceTime         float64
B2_ResponseTime        float64
B2_InterArrivalTime    float64
B2_Throughput          float64
B2_m                   float64
B2_TimestampDiff       float64
B3_Cpu                 float64
B3_Memory              float64
B3_QueueLength         float64
B3_QueueTime           float64
B3_ServiceTime         float64
B3_ResponseTime        float64
B3_InterArrivalTime    float64
B3_Throughput          float64
B3_m    

In [52]:
# hop_df의 time 컬럼의 중복되는 값과 그 개수 확인
# hop_df['time'].value_counts()
hop_df['time'].duplicated().sum()

280

1초에 2개 이상의 행이 존재할 때\
0을 제외한 값들끼리의 평균값(또는 최댓값)을 구해서 하나의 date에는 하나의 행만 남김

In [53]:
hop_df = aggregate_rows_by_time.handle_grouped_duplicate_time(hop_df, method='mean')


In [54]:
# hop_df의 time 컬럼의 중복값 확인
hop_df['time'].duplicated().sum()

0

In [55]:
# 파일 저장
# 디렉토리가 없으면 생성
if not os.path.exists(path_mergeddf):
    os.makedirs(path_mergeddf)

hop_df.to_csv(path_mergeddf + 'hop_df_agg.csv', index=False)

In [56]:
hop_df

,time,B1_Cpu,B1_Memory,B1_QueueLength,B1_QueueTime,B1_ServiceTime,B1_ResponseTime,B1_InterArrivalTime,B1_Throughput,B1_m,B1_TimestampDiff,B2_Cpu,B2_Memory,B2_QueueLength,B2_QueueTime,B2_ServiceTime,B2_ResponseTime,B2_InterArrivalTime,B2_Throughput,B2_m,B2_TimestampDiff,B3_Cpu,B3_Memory,B3_QueueLength,B3_QueueTime,B3_ServiceTime,B3_ResponseTime,B3_InterArrivalTime,B3_Throughput,B3_m,B3_TimestampDiff,B4_Cpu,B4_Memory,B4_QueueLength,B4_QueueTime,B4_ServiceTime,B4_ResponseTime,B4_InterArrivalTime,B4_Throughput,B4_m,B4_TimestampDiff,B5_Cpu,B5_Memory,B5_QueueLength,B5_QueueTime,B5_ServiceTime,B5_ResponseTime,B5_InterArrivalTime,B5_Throughput,B5_m,B5_TimestampDiff
0,2024-11-10 18:10:22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.342480,23302144.0,0.0,0.300874,0.000000,0.300874,0.000000,3.323645,6.682471,NaN,58.481087,22659072.0,0.0,0.109688,0.000000,0.109688,0.000000,9.116759,5.707780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-10 18:10:24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.639821,23822336.0,0.0,0.304711,0.932874,1.237585,1.174595,0.808025,9.035497,NaN,61.388747,23334912.0,0.0,0.111847,0.110225,0.222072,0.774395,4.503044,8.033125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-11-10 18:10:27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.944845,24092672.0,0.0,0.323526,0.986176,1.309703,1.580680,0.763532,11.447164,NaN,66.056112,23732224.0,0.0,0.109257,0.105346,0.214603,0.942848,4.659758,10.421592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-11-10 18:10:29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.767774,24530944.0,0.0,0.303518,1.139734,1.443252,1.835172,0.692880,14.025803,NaN,65.593299,24068096.0,0.0,0.111773,0.120592,0.232365,1.034128,4.303582,12.948760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-11-10 18:10:32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.645620,24924160.0,0.0,0.317207,1.185807,1.503013,2.053788,0.665330,16.967743,NaN,58.115360,24485888.0,0.0,0.106496,0.118152,0.224648,1.118635,4.451416,15.772307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,2024-11-10 20:09:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.377076,71446528.0,0.0,0.014223,1.788314,1.802537,2.048900,0.554774,7173.449767,NaN,57.389806,134955008.0,71414.0,6525.396436,2.354128,6527.750564,0.041454,0.000153,7171.015352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3218,2024-11-10 20:09:54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.379479,41439232.0,0.0,0.001527,2.444717,2.446244,2.049682,0.408790,7178.221656,NaN,57.395523,144584704.0,71412.0,6529.669858,2.183997,6531.853855,0.041454,0.000153,7175.364701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3219,2024-11-10 20:10:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.387428,42385408.0,0.0,0.016074,1.664173,1.680248,2.052395,0.595150,7189.778706,NaN,57.409469,151109632.0,71391.0,6541.819189,0.109411,6541.928600,0.041454,0.000153,7187.828724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3220,2024-11-10 20:10:09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.381996,53686272.0,0.0,0.001001,1.846439,1.847440,2.053005,0.541289,7193.954552,NaN,57.408986,144445440.0,71383.0,6545.737236,0.108261,6545.845497,0.041454,0.000153,7191.917335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Tick Log 전처리

In [57]:
# json 파일을 열고 각 줄을 개별적으로 파싱
# path = '../raw_data/tickLogger/116/'
data = []
with open(path_ticklog + f'{BROKER_PORT}_tick.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

# 데이터를 DataFrame으로 변환
tick_df_raw = pd.DataFrame(data)
type(tick_df_raw)

pandas.core.frame.DataFrame

In [58]:
# time 컬럼의 중복값과 그 개수 확인

duplicates = tick_df_raw['time'].value_counts()

# 중복이 있는 경우만 출력 (카운트가 1보다 큰 경우)
duplicates = duplicates[duplicates > 1]

print("중복된 time 값과 그 개수:")
print(duplicates)

# 전체 중복 레코드 수 확인
total_duplicates = len(tick_df_raw) - len(tick_df_raw['time'].unique())
print(f"\n총 중복 레코드 수: {total_duplicates}")


중복된 time 값과 그 개수:
Series([], Name: time, dtype: int64)

총 중복 레코드 수: 0


In [59]:
# AverageQueueTime, AverageServiceTime, AverageThroughput, InterArrivalTime, Memory, MessageCount, QueueLength, QueueTime, ResponseTime, ServiceTime, Throughtput, TotalArrival Time 컬럼 값을 float 타입으로 변환
# 여러 컬럼 한 번에 type 변경
tick_df = tick_df_raw.astype({'AverageQueueTime': float, 'AverageServiceTime': float, 'AverageThroughput': float, 'InterArrivalTime': float, 'Memory': float, 'MessageCount': float, 'QueueLength': float, 'QueueTime': float, 'ResponseTime': float, 'ServiceTime': float, 'Throughput': float, 'TotalArrival Time': float})


In [60]:
# 파일 저장
# 디렉토리가 없으면 생성
if not os.path.exists(path_mergeddf):
    os.makedirs(path_mergeddf)

tick_df.to_csv(path_mergeddf + 'tick_df.csv', index=False)

In [61]:
tick_df.dtypes


AverageQueueTime      float64
AverageServiceTime    float64
AverageThroughput     float64
Bottleneck               bool
Cpu                   float64
InterArrivalTime      float64
Memory                float64
MessageCount          float64
QueueLength           float64
QueueTime             float64
ResponseTime          float64
ServiceTime           float64
Throughput            float64
TotalArrival Time     float64
level                  object
msg                    object
time                   object
dtype: object

In [62]:
tick_df

,AverageQueueTime,AverageServiceTime,AverageThroughput,Bottleneck,Cpu,InterArrivalTime,Memory,MessageCount,QueueLength,QueueTime,ResponseTime,ServiceTime,Throughput,TotalArrival Time,level,msg,time
0,0.000000,0.000000,0.000000,False,55.680043,0.000000,21209088.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-10 18:10:16
1,0.000000,0.000000,0.000000,False,59.034152,0.000000,23015424.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-10 18:10:17
2,0.000000,0.000000,0.000000,False,59.476336,0.000000,23134208.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-10 18:10:18
3,0.000000,0.000000,0.000000,False,61.530886,0.000000,23146496.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-10 18:10:19
4,0.000000,0.000000,0.000000,False,63.219116,0.000000,23158784.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-10 18:10:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,5.414762,1.239337,0.150283,False,52.379423,2.053005,56442880.0,3501.0,0.0,0.001001,1.847440,1.846439,0.541289,7187.571955,info,Performance Metrics,2024-11-10 20:10:10
7195,5.414762,1.239417,0.150281,False,52.379935,2.053005,56446976.0,3501.0,0.0,0.001001,1.521277,1.520275,0.657343,7187.571955,info,Performance Metrics,2024-11-10 20:10:11
7196,5.414762,1.239417,0.150281,False,52.379832,2.053005,39804928.0,3501.0,0.0,0.001001,1.521277,1.520275,0.657343,7187.571955,info,Performance Metrics,2024-11-10 20:10:12
7197,5.413221,1.239417,0.150316,False,52.378398,2.053518,42283008.0,3502.0,0.0,0.016013,1.536288,1.520275,0.650920,7191.421621,info,Performance Metrics,2024-11-10 20:10:13


In [63]:
merged_df = merge_hop_tick.merge_dataframes(hop_df, tick_df, BROKER_PORT)

In [64]:
merged_df

,AverageQueueTime,AverageServiceTime,AverageThroughput,Bottleneck,B2_Cpu,B2_InterArrivalTime,B2_Memory,MessageCount,B2_QueueLength,B2_QueueTime,B2_ResponseTime,B2_ServiceTime,B2_Throughput,TotalArrival Time,level,msg,time,B1_Cpu,B1_Memory,B1_QueueLength,B1_QueueTime,B1_ServiceTime,B1_ResponseTime,B1_InterArrivalTime,B1_Throughput,B1_m,B1_TimestampDiff,B2_m,B2_TimestampDiff,B3_Cpu,B3_Memory,B3_QueueLength,B3_QueueTime,B3_ServiceTime,B3_ResponseTime,B3_InterArrivalTime,B3_Throughput,B3_m,B3_TimestampDiff,B4_Cpu,B4_Memory,B4_QueueLength,B4_QueueTime,B4_ServiceTime,B4_ResponseTime,B4_InterArrivalTime,B4_Throughput,B4_m,B4_TimestampDiff,B5_Cpu,B5_Memory,B5_QueueLength,B5_QueueTime,B5_ServiceTime,B5_ResponseTime,B5_InterArrivalTime,B5_Throughput,B5_m,B5_TimestampDiff
0,0.000000,0.000000,0.000000,False,55.680043,0.000000,21209088.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-10 18:10:16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000000,0.000000,0.000000,False,59.034152,0.000000,23015424.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-10 18:10:17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000000,0.000000,0.000000,False,59.476336,0.000000,23134208.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-10 18:10:18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000000,0.000000,0.000000,False,61.530886,0.000000,23146496.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-10 18:10:19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.000000,0.000000,0.000000,False,63.219116,0.000000,23158784.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,info,Performance Metrics,2024-11-10 18:10:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,5.414762,1.239337,0.150283,False,52.379423,2.053005,56442880.0,3501.0,0.0,0.001001,1.847440,1.846439,0.541289,7187.571955,info,Performance Metrics,2024-11-10 20:10:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7195,5.414762,1.239417,0.150281,False,52.379935,2.053005,56446976.0,3501.0,0.0,0.001001,1.521277,1.520275,0.657343,7187.571955,info,Performance Metrics,2024-11-10 20:10:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7196,5.414762,1.239417,0.150281,False,52.379832,2.053005,39804928.0,3501.0,0.0,0.001001,1.521277,1.520275,0.657343,7187.571955,info,Performance Metrics,2024-11-10 20:10:12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7197,5.413221,1.239417,0.150316,False,52.379269,2.053518,39804928.0,3502.0,0.0,0.016013,1.536288,1.520275,0.650920,7191.421621,info,Performance Metrics,2024-11-10 20:10:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7197.81923,NaN,57.402841,157093888.0,71381.0,6549.145329,1.629319,6550.774648,0.041454,0.000153,7195.

In [65]:
merged_df['Bottleneck'] = merged_df['Bottleneck'].astype('float64')
merged_df.columns

Index(['AverageQueueTime', 'AverageServiceTime', 'AverageThroughput',
       'Bottleneck', 'B2_Cpu', 'B2_InterArrivalTime', 'B2_Memory',
       'MessageCount', 'B2_QueueLength', 'B2_QueueTime', 'B2_ResponseTime',
       'B2_ServiceTime', 'B2_Throughput', 'TotalArrival Time', 'level', 'msg',
       'time', 'B1_Cpu', 'B1_Memory', 'B1_QueueLength', 'B1_QueueTime',
       'B1_ServiceTime', 'B1_ResponseTime', 'B1_InterArrivalTime',
       'B1_Throughput', 'B1_m', 'B1_TimestampDiff', 'B2_m', 'B2_TimestampDiff',
       'B3_Cpu', 'B3_Memory', 'B3_QueueLength', 'B3_QueueTime',
       'B3_ServiceTime', 'B3_ResponseTime', 'B3_InterArrivalTime',
       'B3_Throughput', 'B3_m', 'B3_TimestampDiff', 'B4_Cpu', 'B4_Memory',
       'B4_QueueLength', 'B4_QueueTime', 'B4_ServiceTime', 'B4_ResponseTime',
       'B4_InterArrivalTime', 'B4_Throughput', 'B4_m', 'B4_TimestampDiff',
       'B5_Cpu', 'B5_Memory', 'B5_QueueLength', 'B5_QueueTime',
       'B5_ServiceTime', 'B5_ResponseTime', 'B5_InterArrivalTime',

In [66]:
# 학습에 불필요한 컬럼 제거
# 불필요한 컬럼 리스트
drop_columns = [
    'level',
    'msg',
    # 'Node',
    # 'TotalArrivalTime',
    'AverageQueueTime',
    'AverageServiceTime',
    'AverageThroughput',
    
    'MessageCount',
    # 'Timestamp'
]

# 불필요한 컬럼 제거
merged_df.drop(columns=drop_columns, inplace=True)

In [67]:
merged_df = merged_df.rename(columns={'time': 'date'})

In [68]:
merged_df

,Bottleneck,B2_Cpu,B2_InterArrivalTime,B2_Memory,B2_QueueLength,B2_QueueTime,B2_ResponseTime,B2_ServiceTime,B2_Throughput,TotalArrival Time,date,B1_Cpu,B1_Memory,B1_QueueLength,B1_QueueTime,B1_ServiceTime,B1_ResponseTime,B1_InterArrivalTime,B1_Throughput,B1_m,B1_TimestampDiff,B2_m,B2_TimestampDiff,B3_Cpu,B3_Memory,B3_QueueLength,B3_QueueTime,B3_ServiceTime,B3_ResponseTime,B3_InterArrivalTime,B3_Throughput,B3_m,B3_TimestampDiff,B4_Cpu,B4_Memory,B4_QueueLength,B4_QueueTime,B4_ServiceTime,B4_ResponseTime,B4_InterArrivalTime,B4_Throughput,B4_m,B4_TimestampDiff,B5_Cpu,B5_Memory,B5_QueueLength,B5_QueueTime,B5_ServiceTime,B5_ResponseTime,B5_InterArrivalTime,B5_Throughput,B5_m,B5_TimestampDiff
0,0.0,55.680043,0.000000,21209088.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-11-10 18:10:16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,59.034152,0.000000,23015424.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-11-10 18:10:17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,59.476336,0.000000,23134208.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-11-10 18:10:18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,61.530886,0.000000,23146496.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-11-10 18:10:19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,63.219116,0.000000,23158784.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2024-11-10 18:10:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,0.0,52.379423,2.053005,56442880.0,0.0,0.001001,1.847440,1.846439,0.541289,7187.571955,2024-11-10 20:10:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7195,0.0,52.379935,2.053005,56446976.0,0.0,0.001001,1.521277,1.520275,0.657343,7187.571955,2024-11-10 20:10:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7196,0.0,52.379832,2.053005,39804928.0,0.0,0.001001,1.521277,1.520275,0.657343,7187.571955,2024-11-10 20:10:12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7197,0.0,52.379269,2.053518,39804928.0,0.0,0.016013,1.536288,1.520275,0.650920,7191.421621,2024-11-10 20:10:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7197.81923,NaN,57.402841,157093888.0,71381.0,6549.145329,1.629319,6550.774648,0.041454,0.000153,7195.327574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
# # 모든 값이 NA인 열들을 확인 후 DataFrame에서 삭제
# print(merged_df.columns[merged_df.isna().all()].tolist())
# merged_df.drop(columns=merged_df.columns[merged_df.isna().all()].tolist(), inplace=True)


In [70]:
# merge_df 의 컬럼 출력
merged_df.columns

Index(['Bottleneck', 'B2_Cpu', 'B2_InterArrivalTime', 'B2_Memory',
       'B2_QueueLength', 'B2_QueueTime', 'B2_ResponseTime', 'B2_ServiceTime',
       'B2_Throughput', 'TotalArrival Time', 'date', 'B1_Cpu', 'B1_Memory',
       'B1_QueueLength', 'B1_QueueTime', 'B1_ServiceTime', 'B1_ResponseTime',
       'B1_InterArrivalTime', 'B1_Throughput', 'B1_m', 'B1_TimestampDiff',
       'B2_m', 'B2_TimestampDiff', 'B3_Cpu', 'B3_Memory', 'B3_QueueLength',
       'B3_QueueTime', 'B3_ServiceTime', 'B3_ResponseTime',
       'B3_InterArrivalTime', 'B3_Throughput', 'B3_m', 'B3_TimestampDiff',
       'B4_Cpu', 'B4_Memory', 'B4_QueueLength', 'B4_QueueTime',
       'B4_ServiceTime', 'B4_ResponseTime', 'B4_InterArrivalTime',
       'B4_Throughput', 'B4_m', 'B4_TimestampDiff', 'B5_Cpu', 'B5_Memory',
       'B5_QueueLength', 'B5_QueueTime', 'B5_ServiceTime', 'B5_ResponseTime',
       'B5_InterArrivalTime', 'B5_Throughput', 'B5_m', 'B5_TimestampDiff'],
      dtype='object')

In [71]:
new_order = ['date', 'Bottleneck',
            #  'HopCount', 
            #  'B1_Timestamp', 'B1_BrokerId',
             'B1_Cpu', 'B1_Memory', 'B1_InterArrivalTime', 'B1_QueueLength', 'B1_QueueTime', 'B1_ServiceTime', 'B1_ResponseTime', 'B1_Throughput',
            #  'B2_Timestamp',  'B2_BrokerId',
             'B2_Cpu', 'B2_Memory', 'B2_InterArrivalTime', 'B2_QueueLength',  'B2_QueueTime', 'B2_ServiceTime', 'B2_ResponseTime', 'B2_Throughput',
            #  'B3_Timestamp', 'B3_BrokerId',
             'B3_Cpu', 'B3_Memory', 'B3_InterArrivalTime', 'B3_QueueLength', 'B3_QueueTime', 'B3_ServiceTime', 'B3_ResponseTime', 'B3_Throughput',
            #  'B4_Timestamp', 'B4_BrokerId',
             'B4_Cpu', 'B4_Memory', 'B4_InterArrivalTime', 'B4_QueueLength', 'B4_QueueTime', 'B4_ServiceTime', 'B4_ResponseTime', 'B4_Throughput',
            #  'B5_Timestamp', 'B5_BrokerId',
             'B5_Cpu', 'B5_Memory', 'B5_InterArrivalTime', 'B5_QueueLength', 'B5_QueueTime', 'B5_ServiceTime', 'B5_ResponseTime', 'B5_Throughput',]

# 새로운 순서로 데이터프레임을 재정렬합니다
merged_df = merged_df[new_order]

In [72]:
drop_columns = [ 
    'Bottleneck',
#     'BrokerId_0',
#     'BrokerId_1',
#     'BrokerId_2'
]

# # 불필요한 컬럼 제거
merged_df.drop(columns=drop_columns, inplace=True)

c:\Users\SJSJ\anaconda3\envs\informer2020\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [73]:
# return all unique values of merged_df columns
for col in merged_df.columns:
    print(col)
    print(merged_df[col].unique())
    print()


date
['2024-11-10 18:10:16' '2024-11-10 18:10:17' '2024-11-10 18:10:18' ...
 '2024-11-10 20:10:12' '2024-11-10 20:10:13' '2024-11-10 20:10:14']

B1_Cpu
[nan]

B1_Memory
[nan]

B1_InterArrivalTime
[nan]

B1_QueueLength
[nan]

B1_QueueTime
[nan]

B1_ServiceTime
[nan]

B1_ResponseTime
[nan]

B1_Throughput
[nan]

B2_Cpu
[55.68004342 59.03415244 59.47633588 ... 52.37983189 52.379269
 52.3755078 ]

B2_Memory
[21209088. 23015424. 23134208. ... 42385408. 76881920. 39804928.]

B2_InterArrivalTime
[0.       1.174595 1.58068  ... 2.052395 2.053005 2.053518]

B2_QueueLength
[ 0.   1.   1.5  3.   4.   2.   5.   6.5  7.   6.   3.5  4.5  2.5  8.
 10.  12.  14.  15.  16.  17.  18.  20.  21.  22.  23.  24.  26.  29.
 30.  31.  33.  34.  36.  37.  39.  38.  40.  41.  43.  45.  46.  48.
 49.  51.  53.  54.  52.5 50.  52.  51.5 50.5 48.5 47.  45.5 44.5 41.5
 39.5 36.5 35.  34.5 32.5 28.  26.5 25.  24.5 18.5 16.5 14.5 13.5 11.
  9.5  7.5  5.5  0.5  9.  13.  19.  32.  42.  44.  46.5 42.5 40.5 37.5
 30.5 29.

In [74]:
# NaN 값을 0으로 대체
merged_df.fillna(0, inplace=True)

c:\Users\SJSJ\anaconda3\envs\informer2020\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [75]:
merged_df

,date,B1_Cpu,B1_Memory,B1_InterArrivalTime,B1_QueueLength,B1_QueueTime,B1_ServiceTime,B1_ResponseTime,B1_Throughput,B2_Cpu,B2_Memory,B2_InterArrivalTime,B2_QueueLength,B2_QueueTime,B2_ServiceTime,B2_ResponseTime,B2_Throughput,B3_Cpu,B3_Memory,B3_InterArrivalTime,B3_QueueLength,B3_QueueTime,B3_ServiceTime,B3_ResponseTime,B3_Throughput,B4_Cpu,B4_Memory,B4_InterArrivalTime,B4_QueueLength,B4_QueueTime,B4_ServiceTime,B4_ResponseTime,B4_Throughput,B5_Cpu,B5_Memory,B5_InterArrivalTime,B5_QueueLength,B5_QueueTime,B5_ServiceTime,B5_ResponseTime,B5_Throughput
0,2024-11-10 18:10:16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55.680043,21209088.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-11-10 18:10:17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.034152,23015424.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-11-10 18:10:18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.476336,23134208.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024-11-10 18:10:19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.530886,23146496.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024-11-10 18:10:20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.219116,23158784.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,2024-11-10 20:10:10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.379423,56442880.0,2.053005,0.0,0.001001,1.846439,1.847440,0.541289,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7195,2024-11-10 20:10:11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.379935,56446976.0,2.053005,0.0,0.001001,1.520275,1.521277,0.657343,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7196,2024-11-10 20:10:12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.379832,39804928.0,2.053005,0.0,0.001001,1.520275,1.521277,0.657343,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7197,2024-11-10 20:10:13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.379269,39804928.0,2.053518,0.0,0.016013,1.520275,1.536288,0.650920,57.402841,157093888.0,0.041454,71381.0,6549.145329,1.629319,6550.774648,0.000153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
# # 각 컬럼별로 unique 값이 하나인 컬럼 삭제
# for col in merged_df.columns:
#     if len(merged_df[col].unique()) == 1:
#         print(col)
#         merged_df = merged_df.drop(col, axis=1)

In [77]:
# save to csv
# 디렉토리가 없으면 생성
if not os.path.exists(path_mergeddf):
    os.makedirs(path_mergeddf)

merged_df.to_csv(path_mergeddf +'merged_df.csv', date_format='%Y-%m-%d %H:%M:%S', index=False)


In [78]:
merged_df.dtypes

date                    object
B1_Cpu                 float64
B1_Memory              float64
B1_InterArrivalTime    float64
B1_QueueLength         float64
B1_QueueTime           float64
B1_ServiceTime         float64
B1_ResponseTime        float64
B1_Throughput          float64
B2_Cpu                 float64
B2_Memory              float64
B2_InterArrivalTime    float64
B2_QueueLength         float64
B2_QueueTime           float64
B2_ServiceTime         float64
B2_ResponseTime        float64
B2_Throughput          float64
B3_Cpu                 float64
B3_Memory              float64
B3_InterArrivalTime    float64
B3_QueueLength         float64
B3_QueueTime           float64
B3_ServiceTime         float64
B3_ResponseTime        float64
B3_Throughput          float64
B4_Cpu                 float64
B4_Memory              float64
B4_InterArrivalTime    float64
B4_QueueLength         float64
B4_QueueTime           float64
B4_ServiceTime         float64
B4_ResponseTime        float64
B4_Throu

In [79]:
merged_df.columns

Index(['date', 'B1_Cpu', 'B1_Memory', 'B1_InterArrivalTime', 'B1_QueueLength',
       'B1_QueueTime', 'B1_ServiceTime', 'B1_ResponseTime', 'B1_Throughput',
       'B2_Cpu', 'B2_Memory', 'B2_InterArrivalTime', 'B2_QueueLength',
       'B2_QueueTime', 'B2_ServiceTime', 'B2_ResponseTime', 'B2_Throughput',
       'B3_Cpu', 'B3_Memory', 'B3_InterArrivalTime', 'B3_QueueLength',
       'B3_QueueTime', 'B3_ServiceTime', 'B3_ResponseTime', 'B3_Throughput',
       'B4_Cpu', 'B4_Memory', 'B4_InterArrivalTime', 'B4_QueueLength',
       'B4_QueueTime', 'B4_ServiceTime', 'B4_ResponseTime', 'B4_Throughput',
       'B5_Cpu', 'B5_Memory', 'B5_InterArrivalTime', 'B5_QueueLength',
       'B5_QueueTime', 'B5_ServiceTime', 'B5_ResponseTime', 'B5_Throughput'],
      dtype='object')

In [83]:
# date 컬럼의 중복값과 그 개수 확인

duplicates = merged_df['date'].value_counts()

# 중복이 있는 경우만 출력 (카운트가 1보다 큰 경우)
duplicates = duplicates[duplicates > 1]

print("중복된 date 값과 그 개수:")
print(duplicates)

# 전체 중복 레코드 수 확인
total_duplicates = len(merged_df) - len(merged_df['date'].unique())
print(f"\n총 중복 레코드 수: {total_duplicates}")


중복된 time 값과 그 개수:
Series([], Name: date, dtype: int64)

총 중복 레코드 수: 0


---

## Temporal-Spatio dataset 구축

In [80]:
# # 사용할 feature 선택

# # 해당 리스트에 있는 문자열로 끝나는 컬럼만 선택
# feature_list = ['Cpu', 'Memory', 'InterArrivalTime', 'QueueLength', 'QueueTime', 'ServiceTime', 'ResponseTime', 'Throughput']

# # feature_list에 있는 문자열로 끝나는 컬럼만 선택
# selected_features = merged_df.filter(regex='|'.join(feature_list))

# selected_features
